In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/Club_Performance.csv')

In [ ]:
df

In [ ]:
df.columns

In [5]:
def calc_l1_points(row):
    l1_count = min(row['Level 1s'], 4)
    return l1_count * 100

def calc_l2_points(row):
    l2_total = row['Level 2s'] + row['Add. Level 2s']
    l2_count = min(l2_total, 2)
    return l2_count * 200

def calc_l3_points(row):
    l3_count = min(row['Level 3s'], 2)
    return l3_count * 300

def calc_l4_points(row):
    return 400 if row['Level 4s, Path Completions, or DTM Awards'] >= 1 else 0

def calc_l5_points(row):
    return 500 if row['Add. Level 4s, Path Completions, or DTM award'] >= 1 else 0

def calc_cot_r1_points(row):
    return 200 if row['Off. Trained Round 1'] >= 7 else 0

def calc_cot_r2_points(row):
    return 200 if row['Off. Trained Round 2'] >= 7 else 0



In [6]:
def assign_club_group(row):
    members = row['Active Members']
    if members < 12:
        return 'Group 1'
    elif 12 <= members <= 20:
        return 'Group 2'
    elif 21 <= members <= 40:
        return 'Group 3'
    elif 41 <= members <= 100:
        return 'Group 4'
    else:
        return 'Unknown'


In [7]:
# Apply group label
df['Group'] = df.apply(assign_club_group, axis=1)

# Optional: Add Name and Description as new columns
group_meta = {
    'Group 1': {
        'Name': 'Spark Clubs',
        'Description': 'Small but full of potential, these clubs are just igniting.'
    },
    'Group 2': {
        'Name': 'Rising Stars',
        'Description': 'Gaining traction, these clubs are building energy and cohesion.'
    },
    'Group 3': {
        'Name': 'Powerhouse Clubs',
        'Description': 'Well-established, these clubs thrive on teamwork and synergy.'
    },
    'Group 4': {
        'Name': 'Pinnacle Clubs',
        'Description': 'Large, vibrant clubs at the peak of influence and activity.'
    },
    'Unknown': {
        'Name': 'Undefined',
        'Description': 'Club size not in defined range.'
    }
}

df['Group Name'] = df['Group'].map(lambda g: group_meta[g]['Name'])
df['Group Description'] = df['Group'].map(lambda g: group_meta[g]['Description'])


In [ ]:
df['L1 Points'] = df.apply(calc_l1_points, axis=1)
df['L2 Points'] = df.apply(calc_l2_points, axis=1)
df['L3 Points'] = df.apply(calc_l3_points, axis=1)
df['L4 Points'] = df.apply(calc_l4_points, axis=1)
df['L5 Points'] = df.apply(calc_l5_points, axis=1)
df['COT R1 Points'] = df.apply(calc_cot_r1_points, axis=1)
df['COT R2 Points'] = df.apply(calc_cot_r2_points, axis=1)
df['Total COT Points'] = df['COT R1 Points'] + df['COT R2 Points']

df['Total Club Points'] = df[['L1 Points', 'L2 Points', 'L3 Points', 'L4 Points', 'L5 Points', 'Total COT Points']].sum(axis=1)
df

In [ ]:
# Sort by Group and Total Club Points descending
df_sorted = df.sort_values(['Group', 'Total Club Points'], ascending=[True, False]).copy()

# Assign rank within each group
df_sorted['Group Rank'] = df_sorted.groupby('Group')['Total Club Points'].rank(
    method='dense', ascending=False
).astype(int)


top_5_by_group = df_sorted[df_sorted['Group Rank'] <= 5][[
    'Group', 'Group Name', 'Club Name', 'Active Members', 'Total Club Points', 'Group Rank'
]].reset_index(drop=True)

top_5_by_group

In [ ]:
https://drive.google.com/file/d/1-bc9u9EeHYJNE35c_NkKUusULCsNYk_3/view?usp=drive_link
https://drive.google.com/file/d/1ZdwpDABrGCmffJ8JuIqbjMKWmzTGIbA4/view?usp=drive_link
https://drive.google.com/file/d/1-bc9u9EeHYJNE35c_NkKUusULCsNYk_3/view?usp=drive_link


In [12]:
# 
gsheet_url = f"https://drive.google.com/uc?export=download&id=1-bc9u9EeHYJNE35c_NkKUusULCsNYk_3"
print(gsheet_url)

https://drive.google.com/uc?export=download&id=1-bc9u9EeHYJNE35c_NkKUusULCsNYk_3


In [13]:
import pandas as pd
df = pd.read_csv(gsheet_url)
df

,District,Division,Area,Club Number,Club Name,Club Status,CSP,Mem. Base,Active Members,Net Growth,...,"Level 4s, Path Completions, or DTM Awards","Add. Level 4s, Path Completions, or DTM award",New Members,Add. New Members,Off. Trained Round 1,Off. Trained Round 2,Mem. dues on time Oct,Mem. dues on time Apr,Off. List On Time,Club Distinguished Status
0,91,A,1.0,5774.0,Solent Speakers Club,Active,N,21.0,21.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN
1,91,A,1.0,1279299.0,Hamwic Speakers,Active,N,45.0,46.0,1.0,...,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,NaN
2,91,A,1.0,2685904.0,Spinnaker Speakers,Active,N,15.0,16.0,1.0,...,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,NaN
3,91,A,1.0,7713357.0,Shilling Speakers,Active,N,25.0,26.0,1.0,...,1.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,1.0,NaN
4,91,A,17.0,8471.0,Reading Speakers Club,Active,Y,41.0,43.0,2.0,...,1.0,1.0,2.0,0.0,3.0,0.0,1.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,91,L,56.0,4580487.0,Tottenham Speakers Club,Active,N,9.0,10.0,1.0,...,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN
178,91,L,56.0,5606419.0,Manor House Speakers,Active,N,14.0,14.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN
179,91,L,56.0,5961042.0,Winchmore Hill Speakers,Active,N,24.0,25.0,1.0,...,0.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,1.0,NaN
180,91,L,56.0,28677377.0,Delta Toastmasters,Ineligible,N,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [14]:
df['Level 1s'].sum(), df['Level 2s'].sum()

(np.float64(72.0), np.float64(29.0))